# QVI Store Trial Analysis – Data Prep & Control Store Selection


### ✔️ Monthly aggregation of sales  
### ✔️ Comparison of stores to identify suitable control stores  
### ✔️ Scoring using:  
- Pearson correlation  
- Magnitude distance (normalized inverse distance)  

### ✔️ Expected Outputs:  
- Aggregated monthly dataset  
- Control-store ranking tables  
- Top-5 control candidates per trial store  
- Selected control store (top-1 by correlation)


In [26]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)


In [27]:
# Load raw dataset
data_path = Path("../data/QVI_data.csv")
df = pd.read_csv(data_path)

df.head()


,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


In [28]:
# Clean column names
df.columns = [c.strip() for c in df.columns]

# Convert DATE column
df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')

# Create YearMonth
df['YearMonth'] = df['DATE'].dt.to_period("M").dt.to_timestamp()

df.head()


,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,YearMonth
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,2018-10-01
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,2018-09-01
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget,2019-03-01
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget,2019-03-01
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,2018-11-01


In [29]:
monthly = (
    df.groupby(["STORE_NBR", "YearMonth"])
      .agg(
          total_sales=("TOT_SALES", "sum"),
          total_customers=("LYLTY_CARD_NBR", pd.Series.nunique),
          total_transactions=("TXN_ID", pd.Series.nunique)
      )
      .reset_index()
)

monthly["avg_tx_per_customer"] = round(monthly["total_transactions"] / monthly["total_customers"], 3)

monthly.head()


,STORE_NBR,YearMonth,total_sales,total_customers,total_transactions,avg_tx_per_customer
0,1,2018-07-01,206.9,49,52,1.061
1,1,2018-08-01,176.1,42,43,1.024
2,1,2018-09-01,278.8,59,62,1.051
3,1,2018-10-01,188.1,44,45,1.023
4,1,2018-11-01,192.6,46,47,1.022


In [30]:
out_path = Path("../data/QVI_monthly_aggregated.csv")
monthly.to_csv(out_path, index=False)

out_path


WindowsPath('../data/QVI_monthly_aggregated.csv')

In [31]:
metric_cols = ["total_sales", "total_customers", "avg_tx_per_customer"]
min_months = 6


def correlation_score_df(trial_store, df=monthly):
    trial = df[df.STORE_NBR == trial_store].set_index("YearMonth").sort_index()

    results = []
    for s in df.STORE_NBR.unique():
        if s == trial_store:
            continue

        cand = df[df.STORE_NBR == s].set_index("YearMonth").sort_index()
        common = trial.index.intersection(cand.index)

        if len(common) < min_months:
            continue

        t = trial.loc[common, metric_cols]
        c = cand.loc[common, metric_cols]

        # correlation per metric
        corrs = [t[col].corr(c[col]) for col in metric_cols]
        score = np.nanmean(corrs)

        results.append({
            "store": int(s),
            "score": float(score),
            "common_months": len(common)
        })
    
    return pd.DataFrame(results).sort_values("score", ascending=False)


In [32]:
def magnitude_distance_df(trial_store, df=monthly):
    trial = df[df.STORE_NBR == trial_store].set_index("YearMonth").sort_index()

    rows = []
    for s in df.STORE_NBR.unique():
        if s == trial_store:
            continue

        cand = df[df.STORE_NBR == s].set_index("YearMonth").sort_index()
        common = trial.index.intersection(cand.index)

        if len(common) < min_months:
            continue

        t = trial.loc[common, metric_cols]
        c = cand.loc[common, metric_cols]

        diff = (t - c).abs().sum().sum()

        rows.append({
            "store": int(s),
            "distance": float(diff),
            "common_months": len(common)
        })

    df_dist = pd.DataFrame(rows)

    if df_dist.empty:
        return df_dist

    dmin, dmax = df_dist.distance.min(), df_dist.distance.max()

    if dmax == dmin:
        df_dist["score"] = 1.0
    else:
        df_dist["score"] = 1 - (df_dist.distance - dmin) / (dmax - dmin)

    return df_dist.sort_values("score", ascending=False)


In [33]:
trial_stores = [77, 86, 88]

control_results = {}

for t in trial_stores:
    print(f"\n=== Trial Store {t} ===")
    corr = correlation_score_df(t)
    mag  = magnitude_distance_df(t)

    control_results[t] = {"correlation": corr, "magnitude": mag}

    display(corr.head(5))
    display(mag.head(5))



=== Trial Store 77 ===


c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: in

,store,score,common_months
160,167,0.517900,12
108,115,0.505698,12
68,71,0.483054,12
61,64,0.480380,12
38,41,0.477391,12


,store,distance,common_months,score
43,46,405.786,12,1.000000
223,233,448.407,12,0.997204
204,214,468.295,12,0.995900
181,188,471.793,12,0.995670
169,176,476.157,12,0.995384



=== Trial Store 86 ===


...

,store,distance,common_months,score
148,155,582.703,12,1.000000
219,229,649.629,12,0.993924
102,109,688.503,12,0.990394
217,227,846.243,12,0.976073
59,62,873.137,12,0.973631



=== Trial Store 88 ===


c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\Niphe\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: in

,store,score,common_months
152,159,0.618517,12
193,201,0.616403,12
11,13,0.505411,12
171,178,0.495427,12
184,191,0.470926,12


,store,distance,common_months,score
37,40,1242.568,12,1.000000
227,237,1297.129,12,0.996661
158,165,1573.722,12,0.979735
191,199,1654.882,12,0.974768
216,226,1875.685,12,0.961256


In [34]:
save_dir = Path("../data/")
save_dir.mkdir(exist_ok=True, parents=True)

created = []

for t in trial_stores:
    corr = control_results[t]["correlation"]
    mag  = control_results[t]["magnitude"]

    corr_path = save_dir / f"trial_{t}_correlation_scores.csv"
    mag_path  = save_dir / f"trial_{t}_magnitude_scores.csv"

    corr.to_csv(corr_path, index=False)
    mag.to_csv(mag_path, index=False)

    created.append(corr_path)
    created.append(mag_path)

created


[WindowsPath('../data/trial_77_correlation_scores.csv'),
 WindowsPath('../data/trial_77_magnitude_scores.csv'),
 WindowsPath('../data/trial_86_correlation_scores.csv'),
 WindowsPath('../data/trial_86_magnitude_scores.csv'),
 WindowsPath('../data/trial_88_correlation_scores.csv'),
 WindowsPath('../data/trial_88_magnitude_scores.csv')]

In [35]:
selected = []

for t in trial_stores:
    corr = control_results[t]["correlation"]
    top = corr.iloc[0]["store"]
    selected.append({"trial_store": t, "selected_control": int(top)})

selected_df = pd.DataFrame(selected)
selected_df


,trial_store,selected_control
0,77,167
1,86,159
2,88,159


In [36]:
selected_df.to_csv(Path("../data/selected_controls_by_trial.csv"), index=False)


This notebook successfully:
- Aggregated monthly sales
- Computed similarity between stores
- Ranked all potential control stores
- Extracted the best control store for each trial store
- Saved all outputs for further analysis

